In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.activations import sigmoid

In [ ]:
data_path = '/content/ex3data1.mat'
weight_path = '/content/ex3weights.mat'
data = loadmat(data_path)
weight = loadmat(weight_path)
data.keys(), weight.keys()

(dict_keys(['__header__', '__version__', '__globals__', 'X', 'y']),
 dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2']))

In [ ]:
# 目标：利用前馈神经网络识别手写数字，预先训练好的模型权重在weight中
X = data['X']
y = data['y']
Theta1 = weight['Theta1']
Theta2 = weight['Theta2']
X.shape, y.shape, Theta1.shape, Theta2.shape

((5000, 400), (5000, 1), (25, 401), (10, 26))

In [ ]:
Theta1T = Theta1.transpose()
Theta2T = Theta2.transpose()
W1 = Theta1T[1:,:]
b1 = Theta1T[0,:]
W2 = Theta2T[1:,:]
b2 = Theta2T[0,:]
W1.shape, b1.shape, W2.shape, b2.shape

((400, 25), (25,), (25, 10), (10,))

In [ ]:
# 建立Model
model = Sequential(
    [
        tf.keras.Input (shape = (400,)),
        Dense(25, activation = sigmoid,name = 'L1'),
        Dense(10, activation = sigmoid,name = 'L2')
     ]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L1 (Dense)                  (None, 25)                10025     
                                                                 
 L2 (Dense)                  (None, 10)                260       
                                                                 
Total params: 10285 (40.18 KB)
Trainable params: 10285 (40.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#设置model参数
model.get_layer('L1').set_weights([W1,b1])
model.get_layer('L2').set_weights([W2,b2])

In [ ]:
# normalize data
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)
X_norm = norm_l(X)

In [ ]:
# 预测图片
y_pred = model.predict(X_norm)
y_pred.shape
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.head()

157/157 [==============================] - 1s 6ms/step


,0,1,2,3,4,5,6,7,8,9
0,0.000835,0.000814,0.000804,0.000010,0.034281,0.004400,0.075350,0.000268,0.026714,0.998951
1,0.006616,0.000188,0.000402,0.000084,0.459847,0.001142,0.011746,0.000118,0.002320,0.999010
2,0.001925,0.000814,0.048571,0.000033,0.034910,0.029444,0.002392,0.020903,0.004268,0.990181
3,0.000781,0.001911,0.013101,0.001869,0.002877,0.067493,0.031514,0.001977,0.000051,0.998925
4,0.000156,0.000665,0.006688,0.000410,0.167692,0.000240,0.031743,0.000402,0.001039,0.994031


In [ ]:
# 从y_pred预测数字
y_prob = np.argmax(y_pred, axis = 1)+1
y = y.reshape(-1,)
y_prob = y_prob.reshape(-1,)

score = sum(1 for t,p in zip(y,y_prob) if t == p)/len(y)
print('准确率：', score)


准确率： 0.8256
